# This script demonstrates loading a HMM model and using it. It also shows how to save a modified model for future use

In [1]:
# Dependency libraries
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas seaborn hmmlearn

In [1]:
# Import required dependencies
import pickle
import pandas as pd
import matplotlib.pyplot as plt

from retry_requests import retry
import requests_cache
import warnings
import itertools
import openmeteo_requests

#### Load Validation Dataset

In [2]:
df_districts = pd.read_csv("RwandaDistrictCentroidsLongitude_Latitude.csv")

In [3]:
districts = dict()
for i in range(len(df_districts)):
    districts[df_districts.loc[i, "District"]] = {
        'province': df_districts.loc[i, "Province"],
        'lat':  -df_districts.loc[i, "Latitude"],
        'long': df_districts.loc[i, "Longitude"]
    }

In [4]:
district_names = ['Gatsibo', 'Muhanga', 'Gicumbi', 'Kicukiro', 'Rulindo', 'Nyaruguru',
       'Rwamagana', 'Kamonyi', 'Gasabo', 'Ngororero', 'Rubavu', 'Nyamagabe',
       'Burera', 'Nyabihu', 'Gakenke', 'Nyamasheke', 'Kirehe', 'Nyagatare',
       'Bugesera', 'Karongi', 'Huye', 'Musanze', 'Rusizi', 'Ngoma',
       'Nyarugenge', 'Gisagara', 'Kayonza', 'Ruhango', 'Nyanza', 'Rutsiro']

In [5]:
daily_data = dict()

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
url = "https://archive-api.open-meteo.com/v1/era5"

for district in districts:
    params = {
    	"latitude": districts[district]['lat'],
    	"longitude":   districts[district]['long'],
        "start_date": "2024-01-01", # Forecast Horizon Starts
        "end_date": "2024-04-20", # Forecast Horizon Ends
    	"daily": "rain_sum"
    }
    responses = openmeteo.weather_api(url, params=params)
    
    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    daily = response.Daily()
    
    daily_data[district] = daily.Variables(0).ValuesAsNumpy()
    
date = pd.date_range(
    	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    	freq = pd.Timedelta(seconds = daily.Interval()),
    	inclusive = "left")

validation_data = pd.DataFrame(data = daily_data, index = date)

### Loading the HMM Model

In [24]:
district_based_models = dict()

for district_name in district_names:
    with open(f"models/districts/{district_name}.pkl", "rb") as file: 
        district_based_models[district_name] = pickle.load(file)

In [25]:
validation_data

,Gatsibo,Muhanga,Gicumbi,Kicukiro,Rulindo,Nyaruguru,Rwamagana,Kamonyi,Gasabo,Ngororero,...,Huye,Musanze,Rusizi,Ngoma,Nyarugenge,Gisagara,Kayonza,Ruhango,Nyanza,Rutsiro
2024-01-01 00:00:00+00:00,0.900000,0.800000,8.5,2.2,9.799999,0.700000,3.7,1.9,3.3,21.600000,...,2.500000,3.000000,1.5,5.300000,3.600000,11.3,0.4,4.700000,3.800000,26.900002
2024-01-02 00:00:00+00:00,5.900000,7.200000,5.4,1.8,8.700000,29.300001,7.9,6.8,5.4,11.000000,...,10.500002,2.000000,10.7,4.500000,1.400000,1.2,0.8,9.799999,8.400000,17.900000
2024-01-03 00:00:00+00:00,4.400000,1.500000,6.4,0.8,3.700000,23.700001,6.0,0.7,1.7,0.700000,...,2.400000,0.300000,2.0,4.200000,6.200000,8.4,4.5,1.600000,7.800000,8.300000
2024-01-04 00:00:00+00:00,0.600000,0.400000,1.9,0.6,1.100000,11.700001,0.7,5.1,0.2,12.400000,...,4.600000,0.600000,5.5,0.200000,2.100000,7.5,0.0,9.700000,7.500000,16.400002
2024-01-05 00:00:00+00:00,0.000000,0.100000,0.2,0.2,0.400000,2.900000,0.1,0.5,0.2,2.500000,...,2.200000,0.000000,0.8,0.000000,0.200000,0.5,0.8,0.400000,1.600000,12.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-16 00:00:00+00:00,0.700000,12.900000,3.0,4.2,3.700000,4.900000,3.4,8.1,4.2,4.500000,...,10.000002,0.100000,1.0,5.400000,9.700001,4.9,0.5,6.100000,13.500001,2.100000
2024-04-17 00:00:00+00:00,2.300000,6.700000,4.0,1.2,5.300000,2.800000,1.5,3.5,3.8,6.699999,...,1.700000,1.000000,2.6,1.600000,1.700000,0.8,0.3,4.600000,3.200000,11.300000
2024-04-18 00:00:00+00:00,3.300000,0.900000,2.9,0.6,1.000000,2.700000,2.9,2.1,2.0,2.500000,...,1.300000,0.700000,7.1,1.800000,2.200000,3.3,1.7,2.200000,1.500000,4.400000
2024-04-19 00:00:00+00:00,1.700000,8.599999,1.2,2.1,2.500000,6.200000,0.7,5.5,1.6,6.300000,...,12.400001,3.000000,7.4,1.400000,4.200000,12.4,1.9,12.000000,5.400000,7.099999


### Infering from the HMM Model

#### Inference per district

In [26]:
forecast_states, _ = district_based_models['Gatsibo'].sample(n_samples=365)
# Viterbi
log_probability, forecast_hidden_states = district_based_models['Gatsibo'].decode(
    forecast_states,
    lengths=len(forecast_states),
    algorithm='viterbi'
)

In [27]:
forecast_hidden_states

array([3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [28]:
data = pd.DataFrame(
    data={
        'hidden_states': forecast_hidden_states.flatten(),
        'forecast_states': forecast_states.flatten()
    },
    index=validation_data.index
)

mean_validation_daily_rainfall_sum = validation_data.loc[:, ['Gasabo']].mean(axis=1)

# Model Evaluation
plt.figure(figsize=(12, 6))
plt.plot(data.index, data["hidden_states"], label='hidden_states')
# plt.plot(data.index, data["forecast_states"], label='forecast_states')
plt.plot(data.index, mean_validation_daily_rainfall_sum, label='actual rainfall')

plt.title('Observing the Coherence of the Forecast States and Actual Rainfall in Kicukiro')
plt.xlabel('Date')
plt.ylabel('States')
plt.grid(True)
plt.legend()
plt.show()

ValueError: Length of values (365) does not match length of index (111)

In [ ]:
forecast_states

In [21]:
timeline = pd.date_range('2024-01-01', '2024-06-30')

In [23]:
len(timeline)

182

### Saving a Modified HMM Model for Future Inference

In [ ]:
# Saving the HMM model to file
with open("models/rwanda.pkl", "wb") as file: 
    pickle.dump(markov_model, file)

In [ ]:
districts